In [1]:
import bilby
import gwpopulation as gwpop
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import corner

from astropy import cosmology, units
from scipy.interpolate import interp1d

from bilby.core.prior import Uniform, LogUniform, PriorDict, Constraint
from gwpopulation.models import mass, spin

xp = gwpop.cupy_utils.xp

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
print(xp)

<module 'cupy' from '/home/deepnika.jain/anaconda3/envs/gwspin/lib/python3.8/site-packages/cupy/__init__.py'>


In [3]:
parameter_translator = dict(
    mass_1_det = 'm1_detector_frame_Msun',
    mass_2_det = 'm2_detector_frame_Msun',
    luminosity_distance = 'luminosity_distance_Mpc',
    a_1 = 'spin1',
    a_2 = 'spin2',
    cos_tilt_1 = 'costilt1',
    cos_tilt_2 = 'costilt2')

In [4]:
posteriors = list()

file = 'GWTC-1_sample_release/GW{}_GWTC-1.hdf5'

events = ['150914', '151012', '151226', '170104', '170608', '170729', '170809', '170814', 
          '170818', '170823']

for event in events:
    _posterior = pd.DataFrame()
    with h5py.File(file.format(event), 'r') as ff:
        for my_key, gwtc_key in parameter_translator.items():
            _posterior[my_key] = ff['IMRPhenomPv2_posterior'][gwtc_key]
    posteriors.append(_posterior)

In [5]:
luminosity_distances = np.linspace(1, 10000, 1000)
redshifts = np.array([cosmology.z_at_value(cosmology.Planck15.luminosity_distance, 
                                                   dl * units.Mpc) for dl in luminosity_distances])
dl_to_z = interp1d(luminosity_distances, redshifts)

In [6]:
for posterior in posteriors:
    posterior['redshift'] = dl_to_z(posterior['luminosity_distance'])
    posterior['mass_1'] = posterior['mass_1_det'] / (1 + posterior['redshift'])
    posterior['mass_2'] = posterior['mass_2_det'] / (1 + posterior['redshift'])
    posterior['mass_ratio'] = posterior['mass_2'] / posterior['mass_1']

In [7]:
modelA = bilby.hyper.model.Model([mass.SinglePeakSmoothedMassDistribution(),
                                  spin.independent_spin_magnitude_beta])

### Default Model Parameters 

| $\alpha$ | $m_{max}$ | $m_{min}$ | $\beta_q$ | $\lambda_m$ | $\mu_m$ | $\sigma_m$ | $\delta_m$ | E[a] | Var[a] |   $\chi$ | $\sigma_a$ | 
|----------|-----------|-----------|-----------|-------------|---------|------------|------------|------|-------|----------|------------|
| [-4, 12] | [30, 100] | [5, 10] | [-4, 12] | [0, 1] | [20, 50] | (0, 10] | [0, 10] | [0, 1] | [0, 0.25] | [0, 1] | [0, 4] |

In [8]:
from gwpopulation.conversions import convert_to_beta_parameters

def prior_conversion(parameters):
    parameters = parameters.copy()
    """Wrapper around conversion for prior constraints"""
    for key in ["amax_1", "amax_2"]:
        if key not in parameters:
            parameters[key] = 1
    parameters, _ = convert_to_beta_parameters(parameters)
    return parameters

In [9]:
priorsA = PriorDict(conversion_function = prior_conversion)
          
# mass
priorsA["alpha"] = Uniform(minimum = -4, maximum = 12, latex_label = "$\\alpha$")
priorsA["beta"] = Uniform(minimum = -4, maximum = 12, latex_label = "$\\beta$")
priorsA["mmin"] = Uniform(minimum = 5, maximum = 10, latex_label = "$M_{min}$")
priorsA["mmax"] = Uniform(minimum = 30, maximum = 100, latex_label = "$M_{max}$")
priorsA["lam"] = Uniform(minimum = 0, maximum = 1, latex_label = "$\\lambda$")
priorsA["mpp"] = Uniform(minimum=20, maximum=50, latex_label='$\\mu_{m}$')
priorsA["sigpp"] = Uniform(minimum=0, maximum=10, latex_label='$\\sigma_{m}$') # sigpp != 0
priorsA["delta_m"] = Uniform(minimum=0, maximum=10, latex_label='$\\delta_{m}$') 

# spin magnitude 
priorsA["amax_1"] = 1
priorsA["amax_2"] = 1
priorsA["mu_chi_1"] = Uniform(minimum=0, maximum=1, latex_label = '$\\mu_{\\chi_1}$')
priorsA["mu_chi_2"] = Uniform(minimum=0, maximum=1, latex_label = '$\\mu_{\\chi_2}$')
priorsA["sigma_chi_1"] = Uniform(minimum=0, maximum=0.5, latex_label = '$\\sigma_{\\chi_1}$')
priorsA["sigma_chi_2"] = Uniform(minimum=0, maximum=0.5, latex_label = '$\\sigma_{\\chi_2}$')

# spin orientation
priorsA["xi_spin"] = Uniform(minimum=0, maximum=1, latex_label='$\\xi$')
priorsA["sigma_1"] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{1}$')
priorsA["sigma_2"] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{2}$')

In [10]:
likelihoodA = gwpop.hyperpe.HyperparameterLikelihood(posteriors=posteriors, 
                                                     hyper_prior=modelA,
                                                     conversion_function = convert_to_beta_parameters)

18:17 bilby INFO    : No prior values provided, defaulting to 1.


In [11]:
likelihoodA.parameters.update(priorsA.sample())
likelihoodA.log_likelihood_ratio()

-1.7976931348623157e+308

In [ ]:
resultA = bilby.run_sampler(likelihood = likelihoodA, priors = priorsA, sampler='dynesty', nlive=200, walks = 10,
                            label='d1', outdir = 'd1', clean = True)

18:17 bilby INFO    : Running for label 'd1', output will be saved to 'd1'
18:17 bilby INFO    : Search parameters:
18:17 bilby INFO    :   alpha = Uniform(minimum=-4, maximum=12, name=None, latex_label='$\\alpha$', unit=None, boundary=None)
18:17 bilby INFO    :   beta = Uniform(minimum=-4, maximum=12, name=None, latex_label='$\\beta$', unit=None, boundary=None)
18:17 bilby INFO    :   mmin = Uniform(minimum=5, maximum=10, name=None, latex_label='$M_{min}$', unit=None, boundary=None)
18:17 bilby INFO    :   mmax = Uniform(minimum=30, maximum=100, name=None, latex_label='$M_{max}$', unit=None, boundary=None)
18:17 bilby INFO    :   lam = Uniform(minimum=0, maximum=1, name=None, latex_label='$\\lambda$', unit=None, boundary=None)
18:17 bilby INFO    :   mpp = Uniform(minimum=20, maximum=50, name=None, latex_label='$\\mu_{m}$', unit=None, boundary=None)
18:17 bilby INFO    :   sigpp = Uniform(minimum=0, maximum=10, name=None, latex_label='$\\sigma_{m}$', unit=None, boundary=None)
18:17 b

0it [00:00, ?it/s]

18:17 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 600, 'nlive': 200, 'first_update': None, 'walks': 10, 'npdim': None, 'rstate': None, 'queue_size': 1, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 120, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7fa79d1539d0>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5}
18:17 bilby INFO    : Checkpoint every check_point_delta_t = 600s
18:17 bilby INFO    : Using dynesty version 1.0.1
18:17 bilby INFO    : Generating initial points from the prio

81it [01:53,  2.41s/it, bound:7 nc: 11 ncall:1.2e+03 eff:6.7% logz-ratio=-67.93+/-0.17 dlogz:35.743>0.1] 

18:19 bilby INFO    : Written checkpoint file d1/d1_resume.pickle


...

1940it [16:37:03, 47.68s/it, bound:1789 nc:2969 ncall:3.1e+06 eff:0.1% logz-ratio=-35.31+/-0.22 dlogz:31.091>0.1]

10:54 bilby INFO    : Written checkpoint file d1/d1_resume.pickle


1953it [16:47:20, 51.64s/it, bound:1802 nc:2913 ncall:3.1e+06 eff:0.1% logz-ratio=-35.30+/-0.22 dlogz:31.021>0.1]

11:05 bilby INFO    : Written checkpoint file d1/d1_resume.pickle
11:05 bilby WARNING : singular matrix
11:05 bilby WARNING : Failed to create dynesty state plot at checkpoint
11:05 bilby WARNING : singular matrix
11:05 bilby WARNING : Failed to create dynesty run plot at checkpoint


1964it [16:55:22, 42.63s/it, bound:1813 nc:2003 ncall:3.1e+06 eff:0.1% logz-ratio=-35.30+/-0.22 dlogz:30.962>0.1]

In [1]:
resultA.plot_corner()

NameError: name 'resultA' is not defined

In [ ]:
y_pred = []
for _, sample in result.posterior.sample(1000).iterrows():
    y_pred.append(modelB(time, **{k:sample[k] for k in likelihood.model_parameters.keys()}))
    
median = np.quantile(y_pred, 0.5, axis=0)
lower = np.quantile(y_pred, 0.01, axis=0)
upper = np.quantile(y_pred, 0.99, axis=0)
plt.plot(time, median, color='C1')
plt.fill_between(time, lower, upper, color="C2", alpha=0.5)

plt.plot(time, yobs, 'x')
plt.show()

In [ ]:
sample = resultA.posterior.sample(1000)
sample

In [ ]:
median = np.quantile(y_pred, 0.5, axis=0)
lower = np.quantile(y_pred, 0.01, axis=0)
upper = np.quantile(y_pred, 0.99, axis=0)